In [5]:
run_memelite ls

explore_richard_data.R.ipynb  test_memelite_PY.ipynb  test_memelite_SH.ipynb


In [9]:
run_memelite python -c "from memelite import fimo; print('FIMO works!')"

run_memelite python -c "from memelite import fimo; print('FIMO works')"
FIMO works


In [10]:
run_memelite python -c 'from memelite import fimo; print("FIMO works!")'

FIMO works!


In [19]:
run_memelite python - <<'EOF'
from memelite.io import read_meme
motifs = read_meme("/hpc/group/igvf/kk319/data/jaspar2024/JASPAR2024_CORE_vertebrates_non-redundant.meme")
print([m for m in motifs.keys() if "Runx1" in m])
EOF

['MA0002.3 Runx1']


In [17]:
run_memelite python - <<'EOF'
from memelite import fimo
import pandas as pd

motif_file = "/hpc/group/igvf/kk319/data/jaspar2024/JASPAR2024_CORE_vertebrates_non-redundant.meme"
fasta_file = "/hpc/group/igvf/kk319/repo/Proj_IGVF_BlueSTARR/results/analysis_variant_delta_motif/test_seq.fa"

# Run with relaxed threshold
hits = fimo(motif_file, fasta_file)
df = pd.concat(hits, ignore_index=True)

print(df[df["motif_name"].str.contains("RUNX1")].head(10))
EOF

<stdin>:9: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
Empty DataFrame
Columns: [motif_name, motif_idx, sequence_name, start, end, strand, score, p-value]
Index: []


In [24]:
run_memelite python - <<'EOF'
import numpy as np
from memelite.io import read_meme
from memelite.utils import one_hot_encode
from Bio import SeqIO

MEME = "/hpc/group/igvf/kk319/data/jaspar2024/JASPAR2024_CORE_vertebrates_non-redundant.meme"
FA   = "/hpc/group/igvf/kk319/repo/Proj_IGVF_BlueSTARR/results/analysis_variant_delta_motif/test_seq.fa"

# Load motifs (keys look like "MA0002.3 Runx1")
motifs = read_meme(MEME)
key = "MA0002.3 Runx1"
assert key in motifs, f"{key} not found; available example keys: {list(motifs.keys())[:5]}"

# PWM from JASPAR is (4, W) -> transpose to (W, 4)
motif_pwm = motifs[key].T
motif_len = motif_pwm.shape[0]  # W

def scan_motif_on_sequence(seq_str, pwm_Wx4):
    # one_hot_encode returns (4, L) -> transpose to (L, 4)
    X = one_hot_encode(seq_str).T
    L = X.shape[0]
    W = pwm_Wx4.shape[0]
    scores = []
    for i in range(L - W + 1):
        window = X[i:i+W, :]          # (W, 4)
        s = np.sum(window * pwm_Wx4)  # elementwise + sum
        scores.append(float(s))
    return scores

for rec in SeqIO.parse(FA, "fasta"):
    scores = scan_motif_on_sequence(str(rec.seq), motif_pwm)
    print(f"{rec.id} | length={len(rec.seq)} | motif_len={motif_len} | n_scores={len(scores)}")
    print("scores:", scores)
    print("max:", max(scores) if scores else None)
    print("-"*60)
EOF

seq1 | length=20 | motif_len=9 | n_scores=12
scores: [2.6805, 2.12, 5.726000000000001, 1.5825, 2.344, 2.6445000000000003, 0.8505, 1.8989999999999998, 1.2530000000000001, 0.9884999999999999, 0.7105, 2.3145]
max: 5.726000000000001
------------------------------------------------------------
seq2 | length=21 | motif_len=9 | n_scores=13
scores: [2.5905, 2.2350000000000003, 2.3385, 1.769, 1.438, 2.1094999999999997, 2.194, 3.1265, 2.489, 5.5955, 1.7435, 3.36, 4.484]
max: 5.5955
------------------------------------------------------------


In [ ]:
run_memelite python - <<'EOF'
import numpy as np
from memelite.io import read_meme
from memelite.utils import one_hot_encode
from Bio import SeqIO

MEME = "/hpc/group/igvf/kk319/data/jaspar2024/JASPAR2024_CORE_vertebrates_non-redundant.meme"
FA   = "/hpc/group/igvf/kk319/repo/Proj_IGVF_BlueSTARR/results/analysis_variant_delta_motif/test_seq.fa"

# Load motifs (keys look like "MA0002.3 Runx1")
motifs = read_meme(MEME)
key = "MA0002.3 Runx1"
assert key in motifs, f"{key} not found; available example keys: {list(motifs.keys())[:5]}"

# PWM from JASPAR is (4, W) -> transpose to (W, 4)
motif_pwm = motifs[key].T
motif_len = motif_pwm.shape[0]  # W

def scan_motif_on_sequence(seq_str, pwm_Wx4):
    # one_hot_encode returns (4, L) -> transpose to (L, 4)
    X = one_hot_encode(seq_str).T
    L = X.shape[0]
    W = pwm_Wx4.shape[0]
    scores = []
    for i in range(L - W + 1):
        window = X[i:i+W, :]          # (W, 4)
        s = np.sum(window * pwm_Wx4)  # elementwise + sum
        scores.append(float(s))
    return scores

for rec in SeqIO.parse(FA, "fasta"):
    scores = scan_motif_on_sequence(str(rec.seq), motif_pwm)
    print(f"{rec.id} | length={len(rec.seq)} | motif_len={motif_len} | n_scores={len(scores)}")
    print("scores:", scores)
    print("max:", max(scores) if scores else None)
    print("-"*60)
EOF

In [4]:
run_memelite python - <<'EOF'
import numpy as np
from memelite.io    import read_meme
from memelite.utils import one_hot_encode

# 
txt_fpath_meme = "/hpc/group/igvf/kk319/data/jaspar2024/JASPAR2024_CORE_vertebrates_non-redundant.meme"

# Example usage:
dct_motif_import = read_meme(txt_fpath_meme)
mat_motif_select = dct_motif_import["MA0002.3 Runx1"]

print(type(mat_motif_select))
print(mat_motif_select.shape)
EOF

<class 'numpy.ndarray'>
(4, 9)


In [ ]:
run_memelite python - <<'EOF'
import numpy as np
from memelite.io import read_meme
from memelite.utils import one_hot_encode

def pwm_to_logodds_Wx4(pwm_4xW, bg=(0.25,0.25,0.25,0.25), eps=1e-6):
    pwm = np.clip(pwm_4xW.astype(float), eps, 1.0)
    lod = np.log2(pwm / np.array(bg)[:,None])
    return lod.T  # (W,4)

def scan_motif_vector(seq_str, pwm_Wx4):
    X = one_hot_encode(seq_str).T  # (L,4)
    L, W = X.shape[0], pwm_Wx4.shape[0]
    return [float(np.sum(X[i:i+W,:] * pwm_Wx4)) for i in range(L - W + 1)]

# Example usage:
motifs = read_meme("/path/JASPAR2024_CORE_vertebrates_non-redundant.meme")
pwm_Wx4 = pwm_to_logodds_Wx4(motifs["MA0002.3 Runx1"])
scores = scan_motif_vector("AAATGTGGTAACCCATGCGT", pwm_Wx4)
print(scores)  # vector of per-position scores
EOF